In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import pandas as pd
import pysal as ps
import geopandas as gpd
import libpysal as lps
import sys
import platform
import zipfile
import os
from datetime import datetime
import time

import seaborn as sns
sns.set(style="darkgrid") # set style

print(sys.version_info)
print("Python Version: " + str(platform.python_version()))

/Users/andreabonilla/Cal/Spring2019/Stat222/crime_env/lib/python3.7/site-packages/pysal/model/spvcm/abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)
Python Version: 3.7.3


In [3]:
# modeling packages
from  statsmodels.api import OLS
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from pysal.model import *
from pysal.explore.esda.moran import Moran
from scipy import stats
from pysal.lib.weights.weights import W
from sklearn.linear_model import LassoCV, LassoLarsCV, LassoLarsIC, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble.partial_dependence import plot_partial_dependence, partial_dependence
from pdpbox import info_plots

In [4]:
def analyze_preds_true(preds_y, true_y):
    """Code to analyze MSE, worst errors, plot residuals"""
    resids = preds_y - true_y
    print('Residual distribution:\n',stats.describe(resids))
    print('\nMSE:',mean_squared_error(true_y, preds_y))
    ## plot 1 is just pred vs true y
    sns.scatterplot(x=preds_y, y=true_y)
    plt.xlabel("Predictions")
    plt.ylabel("True")
    plt.title("Preds vs True")
    plt.show()
    ## plot 2 is redisuals
    sns.residplot(x=preds_y, y=resids,
                  lowess=True, 
                  scatter_kws={'alpha': 0.5}, 
                  line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})
    plt.xlabel("Predictions")
    plt.ylabel("Residuals")
    plt.title("Residual Plots")
    plt.show()

In [5]:
## create two train and test files, 
def create_train_test(data, cols, test_year, y_col):
    prev_year = test_year - 1
    X_train_all = data.query('year!=@test_year')[cols]
    y_train_all = data.query('year!=@test_year')[y_col]
    X_train_prevyear = data.query('year==@prev_year')[cols]
    y_train_prevyear = data.query('year==@prev_year')[y_col]
    
    X_test = data.query('year==@test_year')[cols]
    y_test = data.query('year==@test_year')[y_col]
    ## quick checks
    assert X_train_all.shape[0] == len(y_train_all) 
    assert X_train_prevyear.shape[0] == len(y_train_prevyear) 
    assert X_test.shape[0] == len(y_test) 
    assert X_train_all.shape[1] == len(cols) 
    print('There are %s training obs and %s test obs' % (len(y_train_all), len(y_test)))
    
    return(X_train_all, y_train_all, X_train_prevyear, y_train_prevyear, X_test, y_test)



In [6]:
def grid_search(model, param_search, scoring, X_train, Y_train, cv, refit='neg_mean_squared_error'):
    """Function for grid search and return best estimator
    Options:
    Regression
    ‘explained_variance’: metrics.explained_variance_score
    ‘neg_mean_absolute_error’: metrics.mean_absolute_error
    ‘neg_mean_squared_error’: metrics.mean_squared_error
    ‘neg_mean_squared_log_error’: metrics.mean_squared_log_error
    ‘neg_median_absolute_error’: metrics.median_absolute_error
    ‘r2’: metrics.r2_score

    """
    grid = GridSearchCV(estimator=model, cv=cv,
                        param_grid=param_search, scoring = scoring,
                        refit=refit, return_train_score=True, verbose=1) # metric b/c balanced data set

    gs = grid.fit(X_train, Y_train)
    best_model = gs.best_estimator_ # based on roc_auc score
    return(gs, best_model)

In [7]:
def visualize_parameter_relations(df, x_feature_names, y_col, show_percentile=False):
    """Visualize parameter relationships based on info_plots"""
    # replace _ with sspace and prop case for feature name
    summary_dfs_dict = {}
    cleaned_names = [f.replace("_", " ") for f in x_feature_names]
    
    for f, n in zip(x_feature_names, cleaned_names):
        fig, axes, summary_dfs_dict[f] = info_plots.target_plot(df=yearly_data2.query('year!=2017'), 
                                                       feature=f, 
                                                       feature_name=n,
                                                       target=y_col,
                                                       show_percentile=show_percentile)
        plt.show()
    return(summary_dfs_dict)

## Modeling from Common Variables
Convert from weekly to yearly and start there.

In [8]:
malvika_url = "https://raw.githubusercontent.com/malvikarajeev/sfcrimeanalysis/master/weekly_crime_counts_with_varaibles.csv"
addtl_data = pd.read_csv(malvika_url, index_col=0)
addtl_data['GEOID'] = [str(g).rjust(11, '0') for g in addtl_data['GEOID']]
addtl_data.head(5)

,year,GEOID,week,N,N_calls_311,Estimate_Total,prop_rented,prop_male,prop_african_american,prop_under_poverty_level,prop_vacant_houses,prop_stable,racial_index,income_index,age_index,working_class
1,2010,06075010100,1,34,4,3744,0.337502,0.48531,0.936699,0.795406,0.080802,0.199252,565,1564.1202,528.96,0.503472
2,2010,06075010100,2,29,9,3744,0.337502,0.48531,0.936699,0.795406,0.080802,0.199252,565,1564.1202,528.96,0.503472
3,2010,06075010100,3,21,6,3744,0.337502,0.48531,0.936699,0.795406,0.080802,0.199252,565,1564.1202,528.96,0.503472
4,2010,06075010100,4,21,15,3744,0.337502,0.48531,0.936699,0.795406,0.080802,0.199252,565,1564.1202,528.96,0.503472
5,2010,06075010100,5,22,6,3744,0.337502,0.48531,0.936699,0.795406,0.080802,0.199252,565,1564.1202,528.96,0.503472


In [9]:
yearly_agg_data = addtl_data[['year','GEOID','N','N_calls_311']].groupby(['year','GEOID']).sum().reset_index()
yearly_agg_data.head(5)

,year,GEOID,N,N_calls_311
0,2010,06075010100,1604,600
1,2010,06075010200,823,634
2,2010,06075010300,234,445
3,2010,06075010400,289,599
4,2010,06075010500,1096,663


In [10]:
census_vars = addtl_data.drop(['week','N','N_calls_311'], axis=1).groupby(['year','GEOID']).mean().reset_index()
census_vars.head(5)

,year,GEOID,Estimate_Total,prop_rented,prop_male,prop_african_american,prop_under_poverty_level,prop_vacant_houses,prop_stable,racial_index,income_index,age_index,working_class
0,2010,06075010100,3744,0.337502,0.485310,0.936699,0.795406,0.080802,0.199252,565,1564.120200,528.96,0.503472
1,2010,06075010200,4184,0.362643,0.473948,0.885038,0.950765,0.438360,0.173757,444,2667.206779,747.13,0.352294
2,2010,06075010300,4285,0.354192,0.501750,0.944457,0.808401,0.213598,0.275613,109,1868.883223,648.90,0.415169
3,2010,06075010400,4154,0.377667,0.469186,0.931873,0.949446,0.315683,0.206548,122,2640.336721,750.38,0.422725
4,2010,06075010500,2429,0.378550,0.500206,0.942775,0.940716,0.286248,0.217373,45,1513.915619,458.53,0.306299


In [11]:
yearly_data2 = yearly_agg_data.merge(census_vars, left_on=['year','GEOID'], right_on=['year','GEOID'], how='inner')
yearly_data2.head(5)

,year,GEOID,N,N_calls_311,Estimate_Total,prop_rented,prop_male,prop_african_american,prop_under_poverty_level,prop_vacant_houses,prop_stable,racial_index,income_index,age_index,working_class
0,2010,06075010100,1604,600,3744,0.337502,0.485310,0.936699,0.795406,0.080802,0.199252,565,1564.120200,528.96,0.503472
1,2010,06075010200,823,634,4184,0.362643,0.473948,0.885038,0.950765,0.438360,0.173757,444,2667.206779,747.13,0.352294
2,2010,06075010300,234,445,4285,0.354192,0.501750,0.944457,0.808401,0.213598,0.275613,109,1868.883223,648.90,0.415169
3,2010,06075010400,289,599,4154,0.377667,0.469186,0.931873,0.949446,0.315683,0.206548,122,2640.336721,750.38,0.422725
4,2010,06075010500,1096,663,2429,0.378550,0.500206,0.942775,0.940716,0.286248,0.217373,45,1513.915619,458.53,0.306299


In [12]:
yearly_data2.tail(5)

,year,GEOID,N,N_calls_311,Estimate_Total,prop_rented,prop_male,prop_african_american,prop_under_poverty_level,prop_vacant_houses,prop_stable,racial_index,income_index,age_index,working_class
1555,2017,06075980200,35,137,304,0.395973,0.792763,0.967105,0.959596,0.724832,0.194079,58,64.346717,81.24,0.601974
1556,2017,06075980300,1953,4119,84,0.333333,0.607143,1.000000,1.000000,0.000000,0.261905,0,59.396970,24.91,0.261905
1557,2017,06075980501,235,1951,782,0.343558,0.434783,0.812020,0.632992,0.260123,0.083120,270,147.078210,45.93,0.588235
1558,2017,06075980600,66,211,513,0.403651,0.567251,0.826511,0.623782,1.484787,0.120858,235,89.802561,82.28,0.493177
1559,2017,06075980900,1797,7159,242,0.374815,0.557851,0.760331,0.826446,0.091852,0.152893,21,111.722871,60.36,0.355372


In [13]:
yearly_data2['rate'] = yearly_data2['N'] / yearly_data2['Estimate_Total']
yearly_data2['log_count'] = np.log(yearly_data2['N'])
yearly_data2['log_rate'] = np.log(yearly_data2['rate'])
yearly_data2.head(5)

,year,GEOID,N,N_calls_311,Estimate_Total,prop_rented,prop_male,prop_african_american,prop_under_poverty_level,prop_vacant_houses,prop_stable,racial_index,income_index,age_index,working_class,rate,log_count,log_rate
0,2010,06075010100,1604,600,3744,0.337502,0.485310,0.936699,0.795406,0.080802,0.199252,565,1564.120200,528.96,0.503472,0.428419,7.380256,-0.847654
1,2010,06075010200,823,634,4184,0.362643,0.473948,0.885038,0.950765,0.438360,0.173757,444,2667.206779,747.13,0.352294,0.196702,6.712956,-1.626067
2,2010,06075010300,234,445,4285,0.354192,0.501750,0.944457,0.808401,0.213598,0.275613,109,1868.883223,648.90,0.415169,0.054609,5.455321,-2.907555
3,2010,06075010400,289,599,4154,0.377667,0.469186,0.931873,0.949446,0.315683,0.206548,122,2640.336721,750.38,0.422725,0.069571,5.666427,-2.665400
4,2010,06075010500,1096,663,2429,0.378550,0.500206,0.942775,0.940716,0.286248,0.217373,45,1513.915619,458.53,0.306299,0.451214,6.999422,-0.795812


In [18]:
shapefile0 = gpd.read_file("tl_2010_06_tract10/tl_2010_06_tract10.shp")
shapefile = shapefile0.query("COUNTYFP10=='075' and ALAND10>0") # grab tracts with land 
shapefile.index = list(range(shapefile.shape[0])) # GEOID10 is match with R 
shapefile.head(5)

,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,NAME10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
0,06,075,016500,06075016500,165,Census Tract 165,G5020,S,370459,0,+37.7741958,-122.4477884,"POLYGON ((-122.446471 37.775802, -122.44478 37..."
1,06,075,016400,06075016400,164,Census Tract 164,G5020,S,309097,0,+37.7750995,-122.4369729,"POLYGON ((-122.44034 37.77658, -122.439844 37...."
2,06,075,016300,06075016300,163,Census Tract 163,G5020,S,245867,0,+37.7760456,-122.4295509,"POLYGON ((-122.429152 37.778007, -122.428909 3..."
3,06,075,016100,06075016100,161,Census Tract 161,G5020,S,368901,0,+37.7799831,-122.4286631,"POLYGON ((-122.428909 37.778039, -122.429152 3..."
4,06,075,016000,06075016000,160,Census Tract 160,G5020,S,158236,0,+37.7823363,-122.4224838,"POLYGON ((-122.420425 37.780583, -122.420336 3..."


In [23]:
### INTPTLAT10	INTPTLON10 to numbers to try and get distance but maybe not..
shapefile['INTPTLAT10'] = shapefile['INTPTLAT10'].astype('float64')
shapefile['INTPTLON10'] = shapefile['INTPTLON10'].astype('float64')

/Users/andreabonilla/Cal/Spring2019/Stat222/crime_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/andreabonilla/Cal/Spring2019/Stat222/crime_env/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
## wont do yet

In [24]:
spatial_census = pd.merge(shapefile, yearly_data2, left_on='GEOID10', right_on='GEOID', how='inner')
spatial_census.head(5)

,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,NAME10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,...,prop_under_poverty_level,prop_vacant_houses,prop_stable,racial_index,income_index,age_index,working_class,rate,log_count,log_rate
0,06,075,016500,06075016500,165,Census Tract 165,G5020,S,370459,0,...,0.892645,0.312413,0.222361,396,2612.052450,931.67,0.500416,0.130923,6.445720,-2.033148
1,06,075,016500,06075016500,165,Census Tract 165,G5020,S,370459,0,...,0.866613,0.274818,0.232052,331,2573.161577,894.45,0.487838,0.121420,6.428105,-2.108498
2,06,075,016500,06075016500,165,Census Tract 165,G5020,S,370459,0,...,0.838458,0.222263,0.173248,516,2397.091988,881.33,0.496903,0.128726,6.499787,-2.050067
3,06,075,016500,06075016500,165,Census Tract 165,G5020,S,370459,0,...,0.850349,0.202259,0.202307,446,2552.655480,906.67,0.471356,0.164681,6.769642,-1.803742
4,06,075,016500,06075016500,165,Census Tract 165,G5020,S,370459,0,...,0.864932,0.223764,0.148609,1071,2915.401259,1019.72,0.487126,0.146881,6.745236,-1.918133


### Create Clusters Based on Neighborhood Characteristics

### KMeans
Scale everything first!

In [28]:
## to do later
x_cols = [col for col in list(spatial_census.select_dtypes(exclude='object')) if col not in ['year','N','rate','log_rate','log_count']]
data_no2017 = spatial_census.query('year!=2017')[x_cols]
list(x_cols)

['ALAND10',
 'AWATER10',
 'INTPTLAT10',
 'INTPTLON10',
 'N_calls_311',
 'Estimate_Total',
 'prop_rented',
 'prop_male',
 'prop_african_american',
 'prop_under_poverty_level',
 'prop_vacant_houses',
 'prop_stable',
 'racial_index',
 'income_index',
 'age_index',
 'working_class']

In [30]:
scaler = StandardScaler()
scaler.fit(data_no2017)
data_no2017 = scaler.transform(data_no2017)

print(data_no2017.shape)

(1365, 16)


/Users/andreabonilla/Cal/Spring2019/Stat222/crime_env/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/andreabonilla/Cal/Spring2019/Stat222/crime_env/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


In [33]:
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering


In [ ]:
clustering_algorithms = (
        ('Single Linkage', single),
        ('Average Linkage', average),
        ('Complete Linkage', complete),
        ('Ward Linkage', ward),
    )

    for name, algorithm in clustering_algorithms: